In [1]:
#libraries
import pandas as pd
import wget

In [2]:
import os, ssl
if (not os.environ.get('PYTHONHTTPSVERIFY', '') and
getattr(ssl, '_create_unverified_context', None)):
    ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
# remove existing files
!rm *.csv

# urls of the files

urls = ['https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv',
        'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv',
        'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv',
        'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_country.csv'
]
    
# download files
for url in urls:
    filename = wget.download(url)

In [4]:
# generate DataFrames
confirmed_global = pd.read_csv('time_series_covid19_confirmed_global.csv')
deaths_global = pd.read_csv('time_series_covid19_deaths_global.csv')
recovered_global = pd.read_csv('time_series_covid19_recovered_global.csv')
covid19_data = pd.read_csv('cases_country.csv')

In [5]:
confirmed_global.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/27/22,2/28/22,3/1/22,3/2/22,3/3/22,3/4/22,3/5/22,3/6/22,3/7/22,3/8/22
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,173395,173659,173879,174073,174214,174214,174331,174582,175000,175353
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,271527,271563,271702,271825,271825,272030,272030,272210,272250,272337
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,264855,264936,265010,265079,265130,265186,265227,265265,265297,265323
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,37999,37999,38165,38249,38342,38434,38434,38434,38620,38710
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,98701,98741,98746,98746,98746,98796,98796,98806,98806,98829


In [6]:
# merging three dataframes based on first for columns as all three have same columns.
dates = confirmed_global.columns[4:]
confirmed_global_long = confirmed_global.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
                                              value_vars=dates, var_name='Date', value_name='Confirmed')

deaths_global_long = deaths_global.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
                                        value_vars=dates, var_name='Date', value_name='Deaths')

recovered_global_long = recovered_global.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'],
                                              value_vars=dates, var_name='Date', value_name='Recovered')

In [7]:
data = pd.merge(left=confirmed_global_long, right=deaths_global_long, 
                how ='left', on=['Province/State', 'Country/Region', 'Date', 'Lat', 'Long'])

data = pd.merge(left=data, right=recovered_global_long, how='left',
                on=['Province/State', 'Country/Region', 'Date', 'Lat', 'Long'])

data['Recovered'] = data['Recovered'].fillna(0)
data['Recovered'] = data['Recovered'].astype('int')


In [8]:
# renaming countries, regions, provinces
# i manually find this
data['Country/Region'] = data['Country/Region'].replace('Korea, South', 'South Korea')
data.loc[data['Country/Region'] == "Taiwan*", "Country/Region"] = "Taiwan"

#Change column name
data = data.rename(columns={"Country/Region": "Country"})
data = data.rename(columns={"Province/State": "State"})

data['State'] = data['State'].fillna('')

#create active case column
data["Active"] = data["Confirmed"]-data["Recovered"]-data["Deaths"]

In [9]:
data.to_csv('novel_corona_virus_dataset.csv', index=False)